In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 


In [ ]:
spark=SparkSession.builder\
.appName('ETL')\
.config("spark.jars",r"C:\Users\OMKHOT\OneDrive - Capgemini\Desktop\ETL_COVID_19\postgresql-42.7.2.jar")\
.config("spark.sql.legacy.timeParserPolicy","LEGACY")\
.getOrCreate()

In [ ]:
spark.sparkContext._jsc.sc().listJars()

In [ ]:
covid_data=spark.read.option('multiline','true').json('extracted_data/covid_data_*.json')

In [ ]:
covid_data=covid_data.select(
    col('country'),
col('cases'),
col('todayCases'),
col('deaths'),
col('todayDeaths'),
col('recovered'),
col('active'),
col('population'),
col('continent'),
from_unixtime(col('updated')/1000).alias('last_updated')    
)

#from_unixtime(input's in seconds) updated column has data in miliseconds (/1000 will convert it into seconds).
#

In [ ]:
covid_data=covid_data.withColumn('last_updated',to_timestamp(col('last_updated'),'yyyy-MM-dd HH:mm:ss'))\
.withColumn('country', col('country').cast('string'))\
.withColumn('cases', col('cases').cast('int'))\
.withColumn('todayCases', col('todayCases').cast('int'))\
.withColumn('deaths', col('deaths').cast('int'))\
.withColumn('todayDeaths', col('todayDeaths').cast('int'))\
.withColumn('recovered', col('recovered').cast('int'))\
.withColumn('active', col('active').cast('int'))\
.withColumn('population', col('population').cast('int'))\
.withColumn('continent', col('continent').cast('string'))\
.withColumn('last_updated', col('last_updated').cast('timestamp')).drop('new_last_updated')

In [ ]:
#will put this data into the postgre 
db_properties={
'user':'postgres',
'password':'postgres',
'driver':'org.postgresql.Driver'    
}

In [ ]:
covid_data.write.jdbc(
        url="jdbc:postgresql://localhost:5432/ETL",
        table="covid",
        mode="append",
        properties=db_properties
)

In [ ]:
spark.stop()